<a href="https://colab.research.google.com/github/ivanoveGabuyo/CPE-201L---CPE-2-B/blob/main/Progress_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from tkinter import messagebox, Toplevel, ttk
from datetime import datetime
import sqlite3
import os
from PIL import Image, ImageTk

DB_NAME = "joans_store.db"

def get_default_stock():
    return {
        "Bear Brand": [10, 50],
        "C2": [20, 35],
        "Eggs": [30, 8],
        "Gardenia": [15, 60],
        "Nescafe": [12, 45],
        "Milo": [25, 25],
        "Robust": [15, 40],
        "Sunsilk": [20, 55],
        "Palmolive": [18, 45],
        "Tattoss": [12, 30],
        "Cheppie": [25, 15],
        "Kopiko Blanca": [30, 20],
        "Energen": [15, 25],
        "555 Tuna": [10, 35],
        "Nissin Ramen": [40, 15],
        "Birch Tree": [20, 50],
        "Greatest White": [15, 60],
        "Tide": [10, 65],
        "Safe Guard": [15, 45]
    }

class JoanStoreDB:
    def __init__(self):
        self.conn = sqlite3.connect(DB_NAME)
        self.cursor = self.conn.cursor()
        self.create_tables()
        self.ensure_admin_user()

    def create_tables(self):
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS users (
                username TEXT PRIMARY KEY,
                password TEXT NOT NULL
            )
        """)
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS inventory (
                username TEXT,
                item TEXT,
                quantity INTEGER,
                price INTEGER,
                PRIMARY KEY (username, item),
                FOREIGN KEY (username) REFERENCES users(username)
            )
        """)
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS sales_history (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                username TEXT,
                receipt TEXT,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (username) REFERENCES users(username)
            )
        """)
        self.conn.commit()

    def ensure_admin_user(self):
        self.cursor.execute("SELECT * FROM users WHERE username=?", ("admin",))
        if not self.cursor.fetchone():
            self.cursor.execute("INSERT INTO users (username, password) VALUES (?,?)", ("admin", "admin"))
            default_stock = get_default_stock()
            for item, (qty, price) in default_stock.items():
                self.cursor.execute(
                    "INSERT INTO inventory (username, item, quantity, price) VALUES (?,?,?,?)",
                    ("admin", item, qty, price)
                )
            self.conn.commit()

    def add_user(self, username, password):
        try:
            self.cursor.execute("INSERT INTO users (username, password) VALUES (?,?)", (username, password))
            default_stock = get_default_stock()
            for item, (qty, price) in default_stock.items():
                self.cursor.execute(
                    "INSERT INTO inventory (username, item, quantity, price) VALUES (?,?,?,?)",
                    (username, item, qty, price)
                )
            self.conn.commit()
            return True
        except sqlite3.IntegrityError:
            return False

    def validate_user(self, username, password):
        self.cursor.execute("SELECT password FROM users WHERE username=?", (username,))
        row = self.cursor.fetchone()
        if row and row[0] == password:
            return True
        return False

    def load_inventory(self, username):
        self.cursor.execute("SELECT item, quantity, price FROM inventory WHERE username=?", (username,))
        rows = self.cursor.fetchall()
        return {item: [qty, price] for item, qty, price in rows}

    def update_inventory_item(self, username, item, quantity, price):
        self.cursor.execute("""
            INSERT INTO inventory (username, item, quantity, price)
            VALUES (?,?,?,?)
            ON CONFLICT(username, item) DO UPDATE SET quantity=excluded.quantity, price=excluded.price
        """, (username, item, quantity, price))
        self.conn.commit()

    def update_inventory_quantity(self, username, item, quantity_delta):
        self.cursor.execute("SELECT quantity FROM inventory WHERE username=? AND item=?", (username, item))
        row = self.cursor.fetchone()
        if row:
            new_qty = row[0] + quantity_delta
            if new_qty < 0:
                new_qty = 0
            self.cursor.execute("UPDATE inventory SET quantity=? WHERE username=? AND item=?", (new_qty, username, item))
            self.conn.commit()

    def save_sales_history(self, username, receipt_text):
        self.cursor.execute("INSERT INTO sales_history (username, receipt) VALUES (?,?)", (username, receipt_text))
        self.conn.commit()

    def load_sales_history(self, username):
        self.cursor.execute("SELECT receipt, timestamp FROM sales_history WHERE username=? ORDER BY timestamp DESC", (username,))
        rows = self.cursor.fetchall()
        return [r[0] for r in rows]

    def close(self):
        self.conn.close()

class HoverButton(tk.Button):
    def __init__(self, master=None, **kw):
        tk.Button.__init__(self, master=master, **kw)
        self.defaultBackground = self["background"]
        self.defaultForeground = self["foreground"]
        self.bind("<Enter>", self.on_enter)
        self.bind("<Leave>", self.on_leave)
    def on_enter(self, e):
        self['background'] = self['activebackground']
        self['foreground'] = self['activeforeground']
    def on_leave(self, e):
        self['background'] = self.defaultBackground
        self['foreground'] = self.defaultForeground

class CashierSystem:
    def __init__(self, root):
        self.root = root
        self.root.title("Joan's Store - Cashier System")
        self.root.state('zoomed')  # Maximized window

        # Colors and styles
        self.bg_color = "#f5f5f7"
        self.text_color = "#222222"
        self.subtitle_color = "#555555"
        self.button_bg = "#007acc"
        self.button_active_bg = "#005f99"
        self.accent_button_bg = "#28a745"
        self.accent_button_active_bg = "#1e7e34"
        self.alert_color = "#dc3545"
        self.header_bg = "#007acc"
        self.receipt_header_bg = "#005f99"
        self.entry_bg = "#ffffff"
        self.entry_fg = "#222222"
        self.font_family = "Segoe UI"

        self.has_bg_image = False
        self.datetime_update_id = None

        self.db = JoanStoreDB()

        self.logged_in_user = None
        self.sales_history = []
        self.current_cart = []
        self.stock_inventory = {}
        self.low_stock_threshold = 5
        self.low_stock_warning_shown = False
        self.try_set_background_image("joans_store_bg.png")

        self.main_frame = tk.Frame(self.root, bg=self.bg_color)
        self.main_frame.place(relx=0.5, rely=0.5, anchor="center", relwidth=0.95, relheight=0.95)

        exit_button = HoverButton(self.root, text="✕", command=self.root.destroy,
                                font=(self.font_family, 16, "bold"),
                                bg=self.alert_color, fg="white",
                                activebackground="#a71d2a", activeforeground="white",
                                bd=0, relief=tk.FLAT)
        exit_button.place(x=10, y=10, width=35, height=35)

        self.login_screen()

    def try_set_background_image(self, image_path):
        try:
            if os.path.exists(image_path):
                img = Image.open(image_path)
                screen_width = self.root.winfo_screenwidth()
                screen_height = self.root.winfo_screenheight()
                self.bg_image = ImageTk.PhotoImage(img.resize((screen_width, screen_height)))
                self.bg_label = tk.Label(self.root, image=self.bg_image)
                self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)
                self.has_bg_image = True
            else:
                self.bg_label = tk.Label(self.root, bg=self.bg_color)
                self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)
        except Exception:
            self.bg_label = tk.Label(self.root, bg=self.bg_color)
            self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)

    def clear_window(self):
        if hasattr(self, 'datetime_update_id') and self.datetime_update_id:
            self.root.after_cancel(self.datetime_update_id)
            self.datetime_update_id = None

        if hasattr(self, 'logo_frame'):
            self.logo_frame.pack_forget()

        container = self.main_frame
        for widget in container.winfo_children():
            if widget != getattr(self, 'logo_frame', None):
                widget.destroy()

    def login_screen(self):
        self.clear_window()

        if not hasattr(self, 'logo_frame'):
            self.logo_frame = tk.Frame(self.main_frame, bg=self.bg_color)
            self.logo_frame.pack(pady=(30,10))

            tk.Label(self.logo_frame, text="Joan's Store", font=(self.font_family, 48, "bold"),
                     fg=self.text_color, bg=self.bg_color).pack()
            tk.Label(self.logo_frame, text="Cashier Sales System", font=(self.font_family, 20),
                     fg=self.subtitle_color, bg=self.bg_color).pack(pady=5)

        container = self.main_frame
        self.login_frame = tk.Frame(container, bg=self.bg_color)
        self.login_frame.pack(pady=40)

        tk.Label(self.login_frame, text="Login", font=(self.font_family, 28, "bold"),
                 fg=self.text_color, bg=self.bg_color).grid(row=0, columnspan=2, pady=(0,20))

        tk.Label(self.login_frame, text="Username:", font=(self.font_family, 18),
                 fg=self.text_color, bg=self.bg_color).grid(row=1, column=0, padx=10, pady=10, sticky="e")

        self.username_entry = tk.Entry(self.login_frame, font=(self.font_family, 18), bg=self.entry_bg, fg=self.entry_fg,
                                       bd=2, relief=tk.GROOVE, width=25)
        self.username_entry.grid(row=1, column=1, pady=10, sticky="w")
        self.username_entry.focus_set()

        tk.Label(self.login_frame, text="Password:", font=(self.font_family, 18),
                 fg=self.text_color, bg=self.bg_color).grid(row=2, column=0, padx=10, pady=10, sticky="e")

        self.password_entry = tk.Entry(self.login_frame, font=(self.font_family, 18), show='*',
                                       bg=self.entry_bg, fg=self.entry_fg, bd=2, relief=tk.GROOVE, width=25)
        self.password_entry.grid(row=2, column=1, pady=10, sticky="w")

        login_btn = HoverButton(self.login_frame, text="Login", font=(self.font_family, 16, "bold"),
                              bg=self.button_bg, fg="white", activebackground=self.button_active_bg,
                              activeforeground="white", command=self.login,
                              bd=0, relief=tk.FLAT, padx=20, pady=8)
        login_btn.grid(row=3, columnspan=2, pady=(20,10), sticky="ew")

        signup_btn = HoverButton(self.login_frame, text="Sign Up", font=(self.font_family, 16, "bold"),
                               bg=self.accent_button_bg, fg="white", activebackground=self.accent_button_active_bg,
                               activeforeground="white", command=self.signup_screen,
                               bd=0, relief=tk.FLAT, padx=20, pady=8)
        signup_btn.grid(row=4, columnspan=2, pady=5, sticky="ew")

    def signup_screen(self):
        self.clear_window()

        if not hasattr(self, 'logo_frame'):
            self.logo_frame = tk.Frame(self.main_frame, bg=self.bg_color)
            self.logo_frame.pack(pady=(30,10))

            tk.Label(self.logo_frame, text="Joan's Store", font=(self.font_family, 48, "bold"),
                     fg=self.text_color, bg=self.bg_color).pack()
            tk.Label(self.logo_frame, text="Cashier Sales System", font=(self.font_family, 20),
                     fg=self.subtitle_color, bg=self.bg_color).pack(pady=5)

        container = self.main_frame
        self.signup_frame = tk.Frame(container, bg=self.bg_color)
        self.signup_frame.pack(pady=40)

        tk.Label(self.signup_frame, text="Sign Up", font=(self.font_family, 28, "bold"),
                 fg=self.text_color, bg=self.bg_color).grid(row=0, columnspan=2, pady=(0,20))

        tk.Label(self.signup_frame, text="Username:", font=(self.font_family, 18),
                 fg=self.text_color, bg=self.bg_color).grid(row=1, column=0, padx=10, pady=10, sticky="e")

        self.new_username = tk.Entry(self.signup_frame, font=(self.font_family, 18), bg=self.entry_bg, fg=self.entry_fg,
                                     bd=2, relief=tk.GROOVE, width=25)
        self.new_username.grid(row=1, column=1, pady=10, sticky="w")
        self.new_username.focus_set()

        tk.Label(self.signup_frame, text="Password:", font=(self.font_family, 18),
                 fg=self.text_color, bg=self.bg_color).grid(row=2, column=0, padx=10, pady=10, sticky="e")

        self.new_password = tk.Entry(self.signup_frame, font=(self.font_family, 18), show='*',
                                     bg=self.entry_bg, fg=self.entry_fg, bd=2, relief=tk.GROOVE, width=25)
        self.new_password.grid(row=2, column=1, pady=10, sticky="w")

        create_btn = HoverButton(self.signup_frame, text="Create Account", font=(self.font_family, 16, "bold"),
                               bg=self.button_bg, fg="white", activebackground=self.button_active_bg,
                               activeforeground="white", command=self.create_account,
                               bd=0, relief=tk.FLAT, padx=20, pady=8)
        create_btn.grid(row=3, columnspan=2, pady=(20,10), sticky="ew")

        back_btn = HoverButton(self.signup_frame, text="Back to Login", font=(self.font_family, 16, "bold"),
                             bg=self.accent_button_bg, fg="white", activebackground=self.accent_button_active_bg,
                             activeforeground="white", command=self.login_screen,
                             bd=0, relief=tk.FLAT, padx=20, pady=8)
        back_btn.grid(row=4, columnspan=2, pady=5, sticky="ew")

    def create_account(self):
        username = self.new_username.get().strip()
        password = self.new_password.get().strip()

        if username == '' or password == '':
            messagebox.showerror("Error", "Please fill in all fields.")
            return

        success = self.db.add_user(username, password)
        if success:
            messagebox.showinfo("Success", "Account created successfully!")
            self.login_screen()
        else:
            messagebox.showerror("Error", "Username already exists.")

    def login(self):
        username = self.username_entry.get().strip()
        password = self.password_entry.get().strip()

        if self.db.validate_user(username, password):
            self.logged_in_user = username
            self.sales_history = self.db.load_sales_history(username)
            self.stock_inventory = self.db.load_inventory(username)
            self.setup_ui()
            self.check_low_stock()
        else:
            messagebox.showerror("Error", "Invalid username or password.")

    def setup_ui(self):
        self.clear_window()
        container = self.main_frame

        # Header Frame
        store_header = tk.Frame(container, bg=self.bg_color)
        store_header.pack(fill="x", pady=(10,5))

        tk.Label(store_header, text="Joan's Store", font=(self.font_family, 36, "bold"),
                 fg=self.text_color, bg=self.bg_color).pack(side=tk.LEFT, padx=20)

        header_frame = tk.Frame(container, bg=self.bg_color)
        header_frame.pack(fill="x", pady=(0,10))

        self.datetime_frame = tk.Frame(header_frame, padx=20, pady=10, bg=self.bg_color)
        self.datetime_frame.pack(side=tk.RIGHT, padx=20)

        self.date_label = tk.Label(self.datetime_frame, font=(self.font_family, 20, "bold"),
                                   fg=self.subtitle_color, bg=self.bg_color)
        self.date_label.pack(anchor="e")

        self.time_label = tk.Label(self.datetime_frame, font=(self.font_family, 28, "bold"),
                                   fg=self.subtitle_color, bg=self.bg_color)
        self.time_label.pack(anchor="e")

        self.update_datetime()

        # Top Frame for Search and Quantity
        top_frame = tk.Frame(container, padx=30, pady=15, bg=self.bg_color)
        top_frame.pack(fill="x")

        tk.Label(top_frame, text="🔍 Search Item:", font=(self.font_family, 18, "bold"),
                 fg=self.text_color, bg=self.bg_color).grid(row=0, column=0, padx=10, sticky="w")

        self.search_var = tk.StringVar()
        self.search_entry = tk.Entry(top_frame, textvariable=self.search_var,
                                     font=(self.font_family, 18), width=35, bd=2, relief="groove",
                                     fg=self.entry_fg, bg=self.entry_bg)
        self.search_entry.grid(row=0, column=1, padx=10, sticky="w")
        self.search_entry.bind("<KeyRelease>", self.show_suggestions)

        self.suggestion_listbox = tk.Listbox(top_frame, font=(self.font_family, 16), height=5, width=35,
                                            bd=2, relief="groove", fg=self.entry_fg, bg=self.entry_bg,
                                            highlightthickness=1, highlightbackground="#ccc")
        self.suggestion_listbox.grid(row=1, column=1, sticky="w", padx=10)
        self.suggestion_listbox.bind("<<ListboxSelect>>", self.fill_item_from_suggestion)
        self.suggestion_listbox.grid_remove()

        tk.Label(top_frame, text="🧲 Quantity:", font=(self.font_family, 18, "bold"),
                 fg=self.text_color, bg=self.bg_color).grid(row=2, column=0, padx=10, pady=10, sticky="w")

        self.quantity_entry = tk.Entry(top_frame, font=(self.font_family, 18), width=10,
                                       bd=2, relief="groove", fg=self.entry_fg, bg=self.entry_bg)
        self.quantity_entry.grid(row=2, column=1, padx=10, sticky="w")

        # Buttons Frame with floating style
        button_frame = tk.Frame(container, bg=self.bg_color, pady=15)
        button_frame.pack(fill="x", pady=10)

        row1_frame = tk.Frame(button_frame, bg=self.bg_color)
        row1_frame.pack(pady=10, fill="x")

        row2_frame = tk.Frame(button_frame, bg=self.bg_color)
        row2_frame.pack(pady=10, fill="x")

        buttons_row1 = [
            ("➕ Add Item", self.add_item),
            ("🧾 Generate Receipt", self.generate_receipt),
            ("📜 Sales History", self.show_sales_history),
            ("🧹 Clear Receipt", self.clear_receipt)
        ]

        buttons_row2 = [
            ("📦 Add Stock", self.add_stock_inventory),
            ("✏️ Edit Price", self.edit_item_price),
            ("📊 Check Stock", self.check_stock),
            ("⚠️ Low Stock", self.show_low_stock_items),
            ("🔒 Logout", self.logout)
        ]

        def create_float_button(parent, text, command, alert=False):
            bg = self.alert_color if alert else self.button_bg
            active_bg = "#a71d2a" if alert else self.button_active_bg
            btn = HoverButton(parent, text=text, font=(self.font_family, 18, "bold"),
                            bg=bg, fg="white", activebackground=active_bg,
                            activeforeground="white", bd=0, relief=tk.FLAT,
                            padx=18, pady=8, command=command)
            btn.pack(side=tk.LEFT, padx=10, expand=True, fill="x")
            return btn

        for (txt, cmd) in buttons_row1:
            create_float_button(row1_frame, txt, cmd)
        for (txt, cmd) in buttons_row2:
            create_float_button(row2_frame, txt, cmd, alert=(txt == "⚠️ Low Stock"))

        # Receipt display with mall-like style
        receipt_frame = tk.Frame(container, bg="white", bd=3, relief=tk.SOLID)
        receipt_frame.pack(fill="both", expand=True, padx=30, pady=10)

        receipt_header = tk.Frame(receipt_frame, bg=self.receipt_header_bg)
        receipt_header.pack(fill="x")

        tk.Label(receipt_header, text="Receipt", font=(self.font_family, 24, "bold"),
                 fg="white", bg=self.receipt_header_bg).pack(side=tk.LEFT, padx=10)

        self.receipt_text = tk.Text(receipt_frame, font=("Consolas", 14), bg="white", fg="black",
                                    bd=0, state=tk.DISABLED, wrap=tk.NONE)
        self.receipt_text.pack(fill="both", expand=True)

        # Add horizontal and vertical scrollbars to receipt_text
        receipt_scroll_y = tk.Scrollbar(self.receipt_text.master, orient=tk.VERTICAL, command=self.receipt_text.yview)
        receipt_scroll_y.pack(side=tk.RIGHT, fill=tk.Y)
        self.receipt_text.config(yscrollcommand=receipt_scroll_y.set)

        receipt_scroll_x = tk.Scrollbar(self.receipt_text.master, orient=tk.HORIZONTAL, command=self.receipt_text.xview)
        receipt_scroll_x.pack(side=tk.BOTTOM, fill=tk.X)
        self.receipt_text.config(xscrollcommand=receipt_scroll_x.set)

        # Status bar
        self.status_label = tk.Label(container, text="", font=(self.font_family, 16),
                                     fg=self.alert_color, bg=self.bg_color)
        self.status_label.pack(pady=10)

        # Clear current cart
        self.current_cart = []

    # The rest of your methods remain the same, but here are some key ones updated for style and functionality:

    def add_item(self):
        item = self.search_var.get().strip()
        qty_str = self.quantity_entry.get().strip()
        if item == "":
            messagebox.showerror("Error", "Please enter an item name.")
            return
        if qty_str == "":
            messagebox.showerror("Error", "Please enter quantity.")
            return
        if item not in self.stock_inventory:
            messagebox.showerror("Error", f"Item '{item}' does not exist in inventory.")
            return
        try:
            qty = int(qty_str)
            if qty <= 0:
                raise ValueError
        except ValueError:
            messagebox.showerror("Error", "Quantity must be a positive integer.")
            return

        stock_qty, price = self.stock_inventory[item]

        if qty > stock_qty:
            messagebox.showerror("Error", f"Insufficient stock. Available: {stock_qty}")
            return

        # Add item to current cart or increase qty if already added
        found = False
        for i, (cart_item, cart_qty, cart_price) in enumerate(self.current_cart):
            if cart_item == item:
                self.current_cart[i] = (cart_item, cart_qty + qty, cart_price)
                found = True
                break
        if not found:
            self.current_cart.append((item, qty, price))

        self.update_receipt_display()
        self.search_var.set("")
        self.quantity_entry.delete(0, tk.END)

    def update_receipt_display(self):
        self.receipt_text.config(state=tk.NORMAL)
        self.receipt_text.delete(1.0, tk.END)

        # Mall-like receipt header with columns aligned
        receipt_header = f"{'Item':<20}{'Qty':>5}{'Price':>10}{'Total':>12}\n"
        receipt_header += "-" * 50 + "\n"
        self.receipt_text.insert(tk.END, receipt_header)

        total_price = 0
        for item, qty, price in self.current_cart:
            line_total = qty * price
            self.receipt_text.insert(tk.END, f"{item:<20}{qty:>5}{price:>10,}{line_total:>12,}\n")
            total_price += line_total

        self.receipt_text.insert(tk.END, "-" * 50 + "\n")
        self.receipt_text.insert(tk.END, f"{'Total:':<35}{total_price:>15,}\n")

        self.receipt_text.config(state=tk.DISABLED)

    def generate_receipt(self):
        if not self.current_cart:
            messagebox.showerror("Error", "No items added to receipt.")
            return

        total_price = sum(qty * price for _, qty, price in self.current_cart)

        now = datetime.now()
        receipt_time = now.strftime("%B %d, %Y %I:%M:%S %p")

        receipt_lines = []
        receipt_lines.append("Joan's Store")
        receipt_lines.append("Cashier Sales System")
        receipt_lines.append(receipt_time)
        receipt_lines.append("-" * 50)
        receipt_lines.append(f"{'Item':<20}{'Qty':>5}{'Price':>10}{'Total':>12}")
        receipt_lines.append("-" * 50)

        for item, qty, price in self.current_cart:
            line_total = qty * price
            receipt_lines.append(f"{item:<20}{qty:>5}{price:>10,}{line_total:>12,}")

        receipt_lines.append("-" * 50)
        receipt_lines.append(f"{'Total:':<35}{total_price:>15,}")

        receipt_text = "\n".join(receipt_lines)

        # Save sales history
        self.db.save_sales_history(self.logged_in_user, receipt_text)

        # Update stock in DB and local inventory
        for item, qty, _ in self.current_cart:
            self.db.update_inventory_quantity(self.logged_in_user, item, -qty)
            self.stock_inventory[item][0] -= qty

        self.sales_history.insert(0, receipt_text)
        self.clear_receipt()
        self.update_receipt_display()
        self.check_low_stock()

        # Show receipt in popup
        self.show_receipt_popup(receipt_text)

    def show_receipt_popup(self, receipt_text):
        receipt_win = Toplevel(self.root)
        receipt_win.title("Receipt")
        receipt_win.geometry("480x600")
        receipt_win.config(bg="white")

        text_widget = tk.Text(receipt_win, font=("Consolas", 14), bg="white", fg="black", bd=3, relief=tk.SOLID)
        text_widget.pack(fill="both", expand=True, padx=10, pady=10)
        text_widget.insert(tk.END, receipt_text)
        text_widget.config(state=tk.DISABLED)

        close_btn = HoverButton(receipt_win, text="Close", font=(self.font_family, 16, "bold"),
                              bg=self.button_bg, fg="white", activebackground=self.button_active_bg,
                              activeforeground="white", command=receipt_win.destroy, bd=0, relief=tk.FLAT,
                              padx=20, pady=8)
        close_btn.pack(pady=10)

    def clear_receipt(self):
        self.current_cart.clear()
        self.receipt_text.config(state=tk.NORMAL)
        self.receipt_text.delete(1.0, tk.END)
        self.receipt_text.config(state=tk.DISABLED)

    # Add stock inventory popup with consistent styling
    def add_stock_inventory(self):
        def add_stock():
            item = item_var.get().strip()
            qty_str = qty_entry.get().strip()
            price_str = price_entry.get().strip()

            if item == "":
                messagebox.showerror("Error", "Please enter item name.")
                return
            if item not in self.stock_inventory:
                messagebox.showerror("Error", "Item does not exist.")
                return
            try:
                qty = int(qty_str)
                price = int(price_str)
                if qty <= 0 or price <= 0:
                    raise ValueError
            except ValueError:
                messagebox.showerror("Error", "Quantity and Price must be positive integers.")
                return

            self.db.update_inventory_item(self.logged_in_user, item, self.stock_inventory[item][0] + qty, price)
            self.stock_inventory[item] = [self.stock_inventory[item][0] + qty, price]
            messagebox.showinfo("Success", f"Added {qty} to {item} at price {price}.")
            add_stock_win.destroy()
            self.check_low_stock()

        add_stock_win = Toplevel(self.root)
        add_stock_win.title("Add Stock")
        add_stock_win.geometry("350x320")
        add_stock_win.config(bg=self.bg_color)

        tk.Label(add_stock_win, text="Add Stock", font=(self.font_family, 20, "bold"),
                 fg=self.text_color, bg=self.bg_color).pack(pady=15)

        tk.Label(add_stock_win, text="Item Name:", font=(self.font_family, 16), fg=self.text_color, bg=self.bg_color).pack(pady=5)
        item_var = tk.StringVar()
        item_entry = tk.Entry(add_stock_win, textvariable=item_var, font=(self.font_family, 16), width=25,
                              bd=2, relief=tk.GROOVE, bg=self.entry_bg, fg=self.entry_fg)
        item_entry.pack()

        tk.Label(add_stock_win, text="Quantity to Add:", font=(self.font_family, 16), fg=self.text_color, bg=self.bg_color).pack(pady=5)
        qty_entry = tk.Entry(add_stock_win, font=(self.font_family, 16), width=25, bd=2, relief=tk.GROOVE,
                             bg=self.entry_bg, fg=self.entry_fg)
        qty_entry.pack()

        tk.Label(add_stock_win, text="Price per Item:", font=(self.font_family, 16), fg=self.text_color, bg=self.bg_color).pack(pady=5)
        price_entry = tk.Entry(add_stock_win, font=(self.font_family, 16), width=25, bd=2, relief=tk.GROOVE,
                               bg=self.entry_bg, fg=self.entry_fg)
        price_entry.pack()

        add_btn = HoverButton(add_stock_win, text="Add", font=(self.font_family, 16, "bold"),
                            bg=self.button_bg, fg="white", activebackground=self.button_active_bg,
                            activeforeground="white", command=add_stock,
                            bd=0, relief=tk.FLAT, padx=20, pady=10)
        add_btn.pack(pady=20)

        low_stock_btn = HoverButton(add_stock_win, text="Show Low Stock", font=(self.font_family, 12, "bold"),
                                  bg=self.alert_color, fg="white", activebackground="#a71d2a",
                                  activeforeground="white", command=self.show_low_stock_items,
                                  bd=0, relief=tk.FLAT, padx=15, pady=6)
        low_stock_btn.pack(pady=5)

    # Implement other methods (edit_item_price, check_stock, show_low_stock_items, check_low_stock, show_low_stock_warning, show_sales_history, logout, on_closing) similarly with consistent styling and HoverButton usage.

    def on_closing(self):
        if messagebox.askokcancel("Quit", "Do you want to quit?"):
            self.db.close()
            self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = CashierSystem(root)
    root.protocol("WM_DELETE_WINDOW", app.on_closing)
    root.mainloop()

TclError: no display name and no $DISPLAY environment variable